In [1]:
from pymongo.errors import DuplicateKeyError
from pymongo import MongoClient
from pymongo import errors
from bs4 import BeautifulSoup
import os
import requests
import pandas as pd
import numpy as np
import mimetypes
import requesocks
import time

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
db = client.projectData
clothes = db.clothes
clothes_cols = db.clothesColors
item_urls = db.itemUrls
clothes_urls = db.clothebsUrls

In [ ]:
session = requesocks.session()
 
session.proxies = {
    'http': 'socks5://127.0.0.1:9150',
    'https': 'socks5://127.0.0.1:9150'
}

#Getting Forever21 Dresses

In [ ]:
def get_fov21_data(link):
    html = requests.get(link).text
    soup = BeautifulSoup(html,'html.parser')
    
    img_data = soup.select('a[rel*=small]')
    img_links = [img_data[i]['rel'][-2].split(',')[0] for i in xrange(len(img_data))]
    
    
    title = soup.select('.product-title')
    if title:
        title = title[0].text
    else:
        print 'Title Error at: ', link
        return False
    
    text_price = soup.select('.product-price')
    if text_price:
        price = float(text_price[0].text.replace('$',""))
    else:
        new_test_price = soup.select(".was-now-price")
        if new_test_price:
            price = float(new_test_price[0].text.replace('$',""))
        else:
            print 'Price Error at: ', link
            return False
    
    return {'link': link, 'title': title, 'price': price, 'img_links': img_links, 'source': 'for21'}

In [ ]:
def get_colors(product_id):
    url = 'http://www.forever21.com/Product/Product.aspx?BR=f21&Category=dress&ProductID=20%s&VariantID='%product_id
    html = requests.get(url, headers = {'User-agent': 'Mozilla/5.0'}).text
    soup = BeautifulSoup(html, 'html.parser')
    colors = soup.select('#ctl00_MainContent_upColorList')
    if not colors:
        return None, None
    all_options = colors[0].select('option[value|=%s]'%product_id)
    dress_colors = []
    color_ids = []
    for option in all_options:
        dress_colors.append(option.text)
        color_ids.append(option['value'][-2:])
    return dress_colors, color_ids
    

In [ ]:
def insert_data(urls, database):
    for i, url in enumerate(urls):
        bad_links = []
        data = get_fov21_data(url)

        if not data:
            bad_links.append(url)
            continue

        database.insert_one(data)
        if i%20 == 0:
            print 'Loaded up to %d' %i
        if i%100 == 0:
            time.sleep(60)

In [ ]:
def insert_images_to_db(data,dress_images):
    for images in dress_images:
        dress_id = images['dress_id']
        img_urls = images['img_urls']
        images = images['images']

        data.update({'_id': dress_id}, {'$set': {'img_urls': img_urls, 'img_file_names': images}})

In [9]:
link = 'http://www.forever21.com/Product/Category.aspx?br=f21&category=dress&pagesize=200&page=%d'

Getting a list of the links to the dress pages

In [10]:
num_pages = 7
count = 0
for j in xrange(num_pages):
    html = session.get(link%(j+1), headers = {'User-agent': 'Mozilla/5.0'}).text
    soup = BeautifulSoup(html, 'html.parser')
    all_urls = soup.select('a[href$=VariantID=]')
    for i in xrange(len(all_urls)):
        item_urls.insert_one({'url': all_urls[i]['href'], 'scr': 'Fov21'})
        count += 1
        if count%20 == 0:
            print "loaded up to", count
        if count%100 == 0:
            time.sleep(60)


loaded up to 20
loaded up to 40
loaded up to 60
loaded up to 80
loaded up to 100
loaded up to 120
loaded up to 140
loaded up to 160
loaded up to 180
loaded up to 200
loaded up to 220
loaded up to 240
loaded up to 260
loaded up to 280
loaded up to 300
loaded up to 320
loaded up to 340
loaded up to 360
loaded up to 380
loaded up to 400
loaded up to 420
loaded up to 440
loaded up to 460
loaded up to 480
loaded up to 500
loaded up to 520
loaded up to 540
loaded up to 560
loaded up to 580
loaded up to 600
loaded up to 620
loaded up to 640
loaded up to 660
loaded up to 680
loaded up to 700
loaded up to 720
loaded up to 740
loaded up to 760
loaded up to 780
loaded up to 800
loaded up to 820
loaded up to 840
loaded up to 860
loaded up to 880
loaded up to 900
loaded up to 920
loaded up to 940
loaded up to 960
loaded up to 980
loaded up to 1000
loaded up to 1020
loaded up to 1040
loaded up to 1060
loaded up to 1080
loaded up to 1100
loaded up to 1120
loaded up to 1140
loaded up to 1160
loaded up

In [ ]:
for url in item_urls.distinct('url'):
    clothes_urls.insert_one({'url': url, 'src': 'fov21'})

In [ ]:
view_types = ['1_front', '2_side', '3_back', '4_full', '5_detail', '6_flat', '7_additional']
dress_data = []
counter = 0
for dress in clothes.find():
    img_url = 'http://www.forever21.com/images/%s_330/%s-%s.jpg'
    dress_url = dress['link']
    product_id = dress_url.split('&')[-2][-8:]
    dress_colors, color_ids = get_colors(product_id)
    if not dress_colors:
        continue
    title = dress['title']
    price = dress['price']
    source = dress['source']
    for (color, color_id) in zip (dress_colors, color_ids):
        img_urls = []
        for view in view_types:
            img_urls.append(img_url%(view,product_id,color_id))
        
        dress_data.append({'link': dress_url, 'title': title, 'price': price, 
                           'source': source, 'color': color, 'img_urls': img_urls})
        
    if counter%10 == 0: print "Loaded %d dresses" %counter
        
    counter += 1
    

In [ ]:
for data in dress_data:
    clothes_cols.insert_one(data)

In [297]:
images = []
image_counter = 0
dress_counter = 0

for dress in test[dress_counter:]:

    good_image_links = []
    dress_images = []
    for url in dress['img_urls']:   
        html = session.get(url)

        if html.status_code == 200:
            image_counter += 1
            good_image_links.append(url)
            filename = 'Data/Images/for21%s.png'%image_counter
            
            if image_counter > 602:
                with open(filename, 'wb') as f:
                    f.write(html.content)
                    
            dress_images.append(filename)                

            
    if not good_image_links:
        print "404 Image Error at: ", dress['link']
        continue

    dress_counter += 1
    
    dress['images'] = dress_images
    dress['img_urls'] = good_image_links
    
    images.append({'dress_id': dress['_id'], 'images': dress_images, 'img_urls': good_image_links})

    if dress_counter%10 == 0:
        print "loaded %d dresses total" %dress_counter
        print "Loaded %d images" %image_counter
        print "-----------------------------\n"

    if dress_counter%100 ==0:
        time.sleep(100)

loaded 960 dresses total
Loaded 3432 images
-----------------------------

loaded 970 dresses total
Loaded 3472 images
-----------------------------

loaded 980 dresses total
Loaded 3517 images
-----------------------------

loaded 990 dresses total
Loaded 3556 images
-----------------------------

loaded 1000 dresses total
Loaded 3602 images
-----------------------------

loaded 1010 dresses total
Loaded 3648 images
-----------------------------

loaded 1020 dresses total
Loaded 3687 images
-----------------------------

loaded 1030 dresses total
Loaded 3723 images
-----------------------------

loaded 1040 dresses total
Loaded 3761 images
-----------------------------

loaded 1050 dresses total
Loaded 3800 images
-----------------------------

loaded 1060 dresses total
Loaded 3840 images
-----------------------------

loaded 1070 dresses total
Loaded 3881 images
-----------------------------

loaded 1080 dresses total
Loaded 3919 images
-----------------------------

loaded 1090 dres

In [ ]:
insert_images_to_db(clothes_cols,images)

#Scrapping Jolly Chic

In [ ]:
import psycopg2
with open('~/Documents/Zipfian/project-app/.db_password.txt', 'rb') as f:
    PASSWORD = f.readline()
conn = psycopg2.connect("dbname='postgres' user='Tyler' host='zipfianproject.co9ijljkg9gd.us-east-1.rds.amazonaws.com' port='5432' password=Volkl320")
cur = conn.cursor()

In [3]:
url = 'http://www.jollychic.com/p/ol-style-solid-lace-front-zipper-bodycon-dress-g150599.html#utm_ref=prod_prs-pc_list_one_5'
html = requests.get(url, headers = {'User-agent': 'Mozilla/5.0'}).text
soup = BeautifulSoup(html, 'html.parser')

In [86]:
def scrape_page(url):
    html = requests.get(url, headers = {'User-agent': 'Mozilla/5.0'}).text
    soup = BeautifulSoup(html, 'html.parser')
    #Link
    link = soup.select('.goods-loading.zoomer_img')[0]['src']
 
    #price
    price = float(soup.select('#J-sku-price')[0].text.split()[-1])

    #name
    name = " ".join(soup.select('.goods-tlt')[0].text.split()[:-2])

    #color
    sample = soup.select('.goods-attrs.mt10')[0].text.lower().split()#[1].lower()
    test = np.argmax(np.array(sample) == 'color')
    color = sample[test+1]

    return {'url': url, 'link': link, 'name': name, 'price':price}

In [133]:
page = 'http://www.jollychic.com/womens-dresses-c6?jsort=011%d-119'
dress_urls = []
num_pages = 42
for i in xrange(1,num_pages,1):
    html = requests.get(page%i, headers = {'User-agent': 'Mozilla/5.0'}).text
    soup = BeautifulSoup(html, 'html.parser')

    all_pages = soup.select('.pro_list_imgbox.categoryTwo-imgbox')
        
    for page_info in all_pages:
        dress_urls.append(page_info.select('a[href]')[0]['href'])

In [155]:
all_info = []
counter = 0
for i,url in enumerate(dress_urls):
    try:
        info = scrape_page(url)
    except:
        continue
    all_info.append(info)
    
    if counter%100 ==0:
        print 'Processed %d dresses'%counter
    counter += 1

Processed 0 dresses
Processed 100 dresses
Processed 200 dresses
Processed 300 dresses
Processed 400 dresses
Processed 500 dresses
Processed 600 dresses
Processed 700 dresses
Processed 800 dresses
Processed 900 dresses
Processed 1000 dresses
Processed 1100 dresses
Processed 1200 dresses
Processed 1300 dresses
Processed 1400 dresses
Processed 1500 dresses
Processed 1600 dresses
Processed 1700 dresses
Processed 1800 dresses
Processed 1900 dresses
Processed 2000 dresses
Processed 2100 dresses
Processed 2200 dresses
Processed 2300 dresses
Processed 2400 dresses
Processed 2500 dresses
Processed 2600 dresses
Processed 2700 dresses
Processed 2800 dresses
Processed 2900 dresses
Processed 3000 dresses
Processed 3100 dresses
Processed 3200 dresses
Processed 3300 dresses
Processed 3400 dresses
Processed 3500 dresses
Processed 3600 dresses
Processed 3700 dresses
Processed 3800 dresses
Processed 3900 dresses
Processed 4000 dresses
Processed 4100 dresses
Processed 4200 dresses
Processed 4300 dresses


In [164]:
fil_names = []
counter = 0
for i,info in enumerate(all_info):
    link = info['link']
    
    html = requests.get(link)
    if html.status_code != 200:
        continue
        
    name = 'Data/Images/jolly%s.jpg'%i
    fil_names.append(name)
    info['file_name'] = name
    with open(name,'wb') as f:
        f.write(html.content)
    if counter%100 == 0:
        print 'Fetched %d images'
    counter += 1

all_info2 = [info for info in all_info if 'file_name' in info]
pickle.dump(all_info2,open('new_imgs.plk','wb'),pickle.HIGHEST_PROTOCOL)

Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
Fetched %d images
